In [1]:
!git clone https://github.com/NinomiyaOsuke/pbl-railway-knowledge-system.git
%cd pbl-railway-knowledge-system
%pip install transformers faiss-cpu fugashi ipadic openai langchain langchain-community langchain-openai >> install.txt

Cloning into 'pbl-railway-knowledge-system'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 275 (delta 52), reused 48 (delta 48), pack-reused 216 (from 1)
Receiving objects: 100% (275/275), 3.21 MiB | 7.92 MiB/s, done.
Resolving deltas: 100% (142/142), done.
/content/pbl-railway-knowledge-system


#作問

In [2]:
import os
import re
import random
import pandas as pd
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

In [3]:
# APIキーの設定
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [4]:
def extract_total_reports(content):
    """報告書の総件数を抽出"""
    match = re.search(r'全(\d+)件の報告書', content)
    if match:
        return int(match.group(1))
    return 0

def display_report_summary(report):
    """報告書の内容をわかりやすく表示"""
    print("\n" + "="*50)
    print("選択された報告書の内容")
    print("="*50)

    report_number = re.search(r'報告書番号: (.+?)\n', report)
    if report_number:
        print(f"\n📄 報告書番号: {report_number.group(1)}")

    print("\n📌 基本情報:")
    basic_info = {
        "発生年月日": None,
        "区分": None,
        "発生場所": None,
        "事業者": None,
        "都道府県": None
    }
    for key in basic_info.keys():
        match = re.search(f'\\*\\*{key}\\*\\*: (.+?)\\n', report)
        if match:
            print(f"- {key}: {match.group(1)}")

    print("\n🚨 事故詳細:")
    accident_details = {
        "事故等種類": None,
        "踏切区分": None,
        "人の死傷": None,
        "死傷者数": None
    }
    for key in accident_details.keys():
        match = re.search(f'\\*\\*{key}\\*\\*: (.+?)\\n', report)
        if match:
            print(f"- {key}: {match.group(1)}")

    print("\n📝 概要:")
    if "#### 概要" in report:
        overview = report.split("#### 概要")[1].split("####")[0].strip()
        print(overview)

    print("\n🔍 原因:")
    if "#### 原因" in report:
        cause = report.split("#### 原因")[1].split("###")[0].strip()
        print(cause)

    print("\n" + "="*50 + "\n")

In [5]:
class AccidentAnalyzer:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4o-mini")

    def identify_relevant_regulations(self, accident_report):
        """事故報告書に関連する省令ファイルを特定"""
        try:
            # syourei.txtから省令情報を読み込む
            with open('data/summary.txt', 'r', encoding='utf-8') as f:
                syourei_content = f.read()

            prompt = f"""
            以下の事故報告書を分析し、最も関連性の高い規定ファイルを特定してください。

            事故報告書:
            {accident_report}

            規定ファイルの候補:
            {syourei_content}

            回答形式：
            選択されたファイル: [01-09.mdまでのファイル名のみ、<>の中身のみを出力、<>は出力しない]
            選択理由:
            [具体的な理由の説明]
            """

            result = self.llm.invoke([HumanMessage(content=prompt)])
            print("\n=== 関連省令の特定結果 ===")
            print(result.content)
            print("=" * 50)
            return result.content
        except Exception as e:
            print(f"Error in identify_relevant_regulations: {str(e)}")
            return None

    def analyze_accident(self, accident_report, regulation_content):
        """事故の詳細分析と改善提案の生成"""
        analysis_prompt = f"""
        以下の事故報告書と関連省令を分析し、詳細な分析と改善提案を行ってください。

        事故報告書:
        {accident_report}

        関連省令:
        {regulation_content}

        以下の形式で分析結果を出力してください：

        ===事故分析===
        1. 直接的な原因:
        - [主要な原因の詳細な分析]
        - [関連する要因の特定]

        2. 背景要因:
        - [組織的な要因]
        - [環境的な要因]
        - [人的要因]

        3. 省令との関連:
        - [関連する省令条項の具体的な指摘]
        - [規定と実態のギャップ分析]

        ===改善提案===
        1. 即時対応が必要な項目:
        - [具体的な改善アクション]
        - [期待される効果]
        - [実施における注意点]

        2. 中長期的な改善案:
        - [システム面での改善]
        - [運用面での改善]
        - [教育・訓練面での改善]

        3. 省令改善の提案:
        - [現行規定の問題点]
        - [改善が必要な条項]
        - [具体的な改正案（省令の該当部分を引用し、省令の文章を考えてください）]

        ===予防措置===
        1. 再発防止策:
        - [具体的な防止策]
        - [実施手順]
        - [効果測定方法]

        2. 水平展開:
        - [類似事故の予防策]
        - [他路線・他社への展開可能性]
        """

        try:
            result = self.llm.invoke([HumanMessage(content=analysis_prompt)])
            print("\n=== 事故分析と改善提案 ===")
            print(result.content)
            print("=" * 50)
            return result.content
        except Exception as e:
            print(f"Error in analyze_accident: {str(e)}")
            return None

    def identify_technical_elements(self, accident_report):
        """事故報告書から関連する技術要素を抽出"""
        tech_prompt = f"""
        以下の事故報告書から、関連する鉄道技術の要素を抽出し分析してください。

        事故報告書:
        {accident_report}

        以下の形式で出力してください：

        ===技術要素の特定===
        1. 主要な技術分野:
        - [技術分野名]
        - [この技術が事故に関連する理由]
        - [技術の具体的な要素]

        2. 関連する技術キーワード:
        - [キーワード1]: [関連性の説明]
        - [キーワード2]: [関連性の説明]
        - [キーワード3]: [関連性の説明]

        3. 技術的な問題点:
        - [問題点1]: [詳細な説明]
        - [問題点2]: [詳細な説明]

        ===学習ポイント===
        この事故事例から学ぶべき技術的なポイント:
        1. [学習ポイント1]
        2. [学習ポイント2]
        3. [学習ポイント3]

        ===関連問題案===
        この事故事例に関連して出題できる技術的な問題:

        問題1（基本的な理解）:
        [問題文]
        [正答]
        [解説]

        問題2（技術的な分析）:
        [問題文]
        [正答]
        [解説]

        問題3（予防的思考）:
        [問題文]
        [正答]
        [解説]

        ===逆引きキーワード===
        以下のキーワードで検索したときに、この事故事例が参考になる：
        - [キーワード1]: [このキーワードでの検索での適合理由]
        - [キーワード2]: [このキーワードでの検索での適合理由]
        - [キーワード3]: [このキーワードでの検索での適合理由]
        """

        try:
            result = self.llm.invoke([HumanMessage(content=tech_prompt)])
            print("\n=== 技術要素の分析結果 ===")
            print(result.content)
            print("=" * 50)
            return result.content
        except Exception as e:
            print(f"Error in identify_technical_elements: {str(e)}")
            return None

    def generate_tech_based_questions(self, accident_report, technical_analysis):
        """技術要素に基づく問題生成"""
        question_prompt = f"""
        以下の事故報告書と技術分析に基づいて、技術的な観点からの問題を生成してください。

        事故報告書:
        {accident_report}

        技術分析:
        {technical_analysis}

        以下の形式で出力してください：

        ===技術習得のための問題===

        【基礎知識確認】
        問題：
        [この事故に関連する基礎的な技術知識を問う問題]
        解答：[解答]
        解説：[技術的な解説]
        関連キーワード：[この問題に関連する技術キーワード]

        【応用問題】
        問題：
        [技術的な判断や分析を必要とする問題]
        解答：[解答]
        解説：[技術的な解説と実務での応用]
        関連キーワード：[この問題に関連する技術キーワード]

        【予防保全的思考】
        問題：
        [類似事故の予防に関する技術的な問題]
        解答：[解答]
        解説：[予防保全の観点からの解説]
        関連キーワード：[この問題に関連する技術キーワード]

        ===この事例が有用な技術者===
        - [職種1]: [この事例が参考になる理由]
        - [職種2]: [この事例が参考になる理由]
        - [職種3]: [この事例が参考になる理由]
        """

        try:
            result = self.llm.invoke([HumanMessage(content=question_prompt)])
            print("\n=== 技術的問題生成結果 ===")
            print(result.content)
            print("=" * 50)
            return result.content
        except Exception as e:
            print(f"Error in generate_tech_based_questions: {str(e)}")
            return None

    def generate_practical_questions(self, accident_report, regulation_content, technical_analysis):
        """実践的な問題と解答の生成"""
        question_prompt = f"""
        以下の事故報告書、関連省令、技術分析に基づいて、実践的な問題と解答を生成してください。
        生成する問題は、実際の事故事例と省令の内容を結びつけ、実務に即した内容としてください。

        事故報告書:
        {accident_report}

        関連省令:
        {regulation_content}

        技術分析:
        {technical_analysis}

        以下の形式で4つの問題を生成してください：

        【問題1：法令と技術の統合的理解】
        対象者: [この問題が適している学習者の属性や役職、例：新人運転士、運行管理者、保線作業員など]
        状況：
        [実務的な状況設定]

        設問：
        1) [省令に基づいた具体的な問題]
        2) [技術的判断を問う問題]

        模範解答：
        設問1:
        [具体的な解答と解説]

        設問2:
        [具体的な解答と解説]

        【問題2：事故分析と予防措置】
        対象者: [この問題が適している学習者]
        [同様の形式]

        【問題3：緊急時対応と法令遵守】
        対象者: [この問題が適している学習者]
        [同様の形式]

        【問題4：設備管理と法令対応】
        対象者: [この問題が適している学習者]
        [同様の形式]

        各問題では以下の要素を必ず含めてください：
        - 実際の事故事例との関連性
        - 関連省令の具体的な適用
        - 実務的な判断
        - 技術的な理解
        - 安全管理の視点
        """

        try:
            result = self.llm.invoke([HumanMessage(content=question_prompt)])
            print("\n=== 実践的な問題と解答の生成結果 ===")
            print(result.content)
            print("=" * 50)
            return result.content
        except Exception as e:
            print(f"Error in generate_practical_questions: {str(e)}")
            return None

    def generate_analysis(self, accident_report):
        print("\n========== 事故分析プロセス開始 ==========")

        relevant_reg_result = self.identify_relevant_regulations(accident_report)
        if not relevant_reg_result:
            return None

        reg_file_match = re.search(r'選択されたファイル:\\s*(\\d+\\.md)', relevant_reg_result)
        if not reg_file_match:
            return None

        regulation_file = reg_file_match.group(1)
        print(f"\n📁 使用する省令ファイル: {regulation_file}")

        try:
            with open(os.path.join('data', regulation_file), 'r', encoding='utf-8') as f:
                regulation_content = f.read()
                print(f"✅ 省令ファイル読み込み完了 ({len(regulation_content)} 文字)")
        except Exception as e:
            print(f"❌ ファイル読み込み時にエラーが発生: {str(e)}")
            return None

        # 事故分析の実行
        analysis_result = self.analyze_accident(accident_report, regulation_content)
        if not analysis_result:
            return None

        # 技術要素の分析
        print("\n🔍 技術要素の分析を開始...")
        technical_analysis = self.identify_technical_elements(accident_report)

        # 省令改善案の生成
        print("\n📋 省令改善案の生成を開始...")
        regulation_improvements = generate_regulation_improvements(
            accident_report,
            regulation_content,
            analysis_result
        )

        if regulation_improvements:
            improvements = parse_regulation_improvements(regulation_improvements)
            display_regulation_improvements(improvements)

            # 改善案をCSVファイルとして保存
            improvements_data = []
            for imp in improvements:
                improvements_data.append({
                    'proposal_number': imp['number'],
                    'target_article': imp['target'],
                    'current_regulation': imp['current'],
                    'improved_regulation': imp['improvement'],
                    'improvement_reason': imp['reason'],
                    'expected_effect': imp['effect']
                })

            if improvements_data:
                df = pd.DataFrame(improvements_data)
                os.makedirs('output', exist_ok=True)
                df.to_csv('output/regulation_improvements.csv', index=False, encoding='utf-8')
                print("\n✅ 省令改善案を生成し保存しました")

        # 実践的な問題の生成
        print("\n📝 実践的な問題と解答の生成を開始...")
        if technical_analysis and regulation_content:
            practical_questions = self.generate_practical_questions(
                accident_report,
                regulation_content,
                technical_analysis
            )

            if practical_questions:
                qa_data = parse_practical_questions(practical_questions)
                qa_list = []
                for problem in qa_data:
                    qa_list.append({
                        'problem_number': problem['number'],
                        'situation': problem['situation'],
                        'questions': '\n'.join(problem['questions']),
                        'answers': '\n'.join(problem['answers'])
                    })
                qa_df = pd.DataFrame(qa_list)
                qa_df.to_csv('output/practical_questions.csv', index=False, encoding='utf-8')
                print("\n✅ 実践的な問題と解答を生成し保存しました")
        else:
            print("❌ 問題生成に必要な情報が不足しています")

        return {
            'analysis': analysis_result,
            'technical_analysis': technical_analysis,
            'practical_questions': practical_questions,
            'regulation_improvements': regulation_improvements
        }

    def generate_regulation_improvements(self, accident_report, regulation_content, analysis_result):
        """事故分析結果から省令の改善案を生成する"""
        improvement_prompt = f"""
        以下の事故報告書、現行省令、事故分析結果に基づいて、省令の具体的な改善案を提示してください。

        事故報告書:
        {accident_report}

        現行省令:
        {regulation_content}

        事故分析結果:
        {analysis_result}

        以下の形式で改善案を提示してください：

        ===省令改善提案===

        【改善提案1】
        対象条文: [該当する条文番号と名称]
        現行規定: [現在の条文内容]
        改善案: [具体的な改正案]
        改善理由: [なぜこの改善が必要か]
        期待効果: [改善後に期待される効果]

        【改善提案2】
        ... (同様の形式で複数の改善案がある場合は続く)
        """

        try:
            result = self.llm.invoke([HumanMessage(content=improvement_prompt)])
            print("\n=== 省令改善案の生成結果 ===")
            print(result.content)
            print("=" * 50)
            return result.content
        except Exception as e:
            print(f"Error in generate_regulation_improvements: {str(e)}")
            return None

In [6]:
def parse_analysis(raw_text):
    """分析結果をパース"""
    analysis_data = {
        "事故分析": {
            "直接的な原因": [],
            "背景要因": [],
            "省令との関連": []
        },
        "改善提案": {
            "即時対応項目": [],
            "中長期的改善案": [],
            "省令改善の提案": []
        },
        "予防措置": {
            "再発防止策": [],
            "水平展開": []
        }
    }

    current_section = None
    current_subsection = None

    lines = raw_text.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('==='):
            current_section = line.strip('=').strip()
            continue

        if line.startswith('1.') or line.startswith('2.') or line.startswith('3.'):
            current_subsection = line.split(':')[0].split('.')[1].strip()
            continue

        if line.startswith('-') and current_section and current_subsection:
            if current_section in analysis_data:
                if current_subsection in analysis_data[current_section]:
                    analysis_data[current_section][current_subsection].append(line.strip('- '))

    return analysis_data


def parse_technical_analysis(raw_text):
    """技術分析結果をパース"""
    tech_data = {
        "技術要素": {
            "主要分野": [],
            "技術キーワード": {},
            "技術的問題点": {}
        },
        "学習ポイント": [],
        "関連問題": [],
        "逆引きキーワード": {}
    }

    current_section = None
    current_subsection = None

    lines = raw_text.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('==='):
            current_section = line.strip('=').strip()
            continue

        if current_section == "技術要素の特定":
            if line.startswith('1. 主要な技術分野'):
                current_subsection = "主要分野"
            elif line.startswith('2. 関連する技術キーワード'):
                current_subsection = "技術キーワード"
            elif line.startswith('3. 技術的な問題点'):
                current_subsection = "技術的問題点"
            elif line.startswith('-'):
                if current_subsection == "主要分野":
                    tech_data["技術要素"]["主要分野"].append(line.strip('- '))
                elif ":" in line:
                    key, value = line.strip('- ').split(':', 1)
                    if current_subsection == "技術キーワード":
                        tech_data["技術要素"]["技術キーワード"][key.strip()] = value.strip()
                    elif current_subsection == "技術的問題点":
                        tech_data["技術要素"]["技術的問題点"][key.strip()] = value.strip()

        elif current_section == "学習ポイント":
            if line.startswith(('1.', '2.', '3.')):
                tech_data["学習ポイント"].append(line.split('.', 1)[1].strip())

        elif current_section == "関連問題案":
            if line.startswith('問題'):
                tech_data["関連問題"].append({
                    "問題番号": line,
                    "問題文": "",
                    "正答": "",
                    "解説": ""
                })
            elif tech_data["関連問題"]:
                current_problem = tech_data["関連問題"][-1]
                if not current_problem["問題文"]:
                    current_problem["問題文"] = line
                elif not current_problem["正答"]:
                    current_problem["正答"] = line
                elif not current_problem["解説"]:
                    current_problem["解説"] = line

        elif current_section == "逆引きキーワード":
            if line.startswith('-') and ":" in line:
                key, value = line.strip('- ').split(':', 1)
                tech_data["逆引きキーワード"][key.strip()] = value.strip()

    return tech_data


def parse_practical_questions(raw_text):
    """実践的な問題と解答をパース"""
    qa_data = []
    current_problem = None
    current_section = None

    lines = raw_text.split('\n')
    for line in lines:
        line = line.strip()

        if line.startswith('【問題'):
            if current_problem:
                qa_data.append(current_problem)
            current_problem = {
                'number': line,
                'target_audience': '',
                'situation': '',
                'questions': [],
                'answers': []
            }
        elif current_problem:
            if line.startswith('対象者:'):
              current_problem['target_audience'] = line[4:].strip()
            elif line.startswith('状況：'):
                current_problem['situation'] = line[3:].strip()
            elif line.startswith('設問'):
                if not line.startswith('設問：'):
                    current_problem['questions'].append(line)
            elif line.startswith('模範解答：'):
                current_section = 'answers'
            elif current_section == 'answers' and line and not line.startswith('【'):
                current_problem['answers'].append(line)

    if current_problem:
        qa_data.append(current_problem)

    return qa_data

In [7]:
def parse_regulation_improvements(improvements_text):
    """省令改善案をパースして構造化データにする"""
    improvements = []
    current_improvement = None

    for line in improvements_text.split('\n'):
        line = line.strip()

        if line.startswith('【改善提案'):
            if current_improvement:
                improvements.append(current_improvement)
            current_improvement = {
                'number': line,
                'target': '',
                'current': '',
                'improvement': '',
                'reason': '',
                'effect': ''
            }
        elif current_improvement:
            if line.startswith('対象条文:'):
                current_improvement['target'] = line.replace('対象条文:', '').strip()
            elif line.startswith('現行規定:'):
                current_improvement['current'] = line.replace('現行規定:', '').strip()
            elif line.startswith('改善案:'):
                current_improvement['improvement'] = line.replace('改善案:', '').strip()
            elif line.startswith('改善理由:'):
                current_improvement['reason'] = line.replace('改善理由:', '').strip()
            elif line.startswith('期待効果:'):
                current_improvement['effect'] = line.replace('期待効果:', '').strip()

    if current_improvement:
        improvements.append(current_improvement)

    return improvements

def display_regulation_improvements(improvements):
    """省令改善案を見やすく表示する"""
    print("\n=== 省令改善提案 ===")

    for imp in improvements:
        print(f"\n{imp['number']}")
        print(f"📋 対象条文: {imp['target']}")
        print("\n📝 規定内容:")
        print(f"  現行: {imp['current']}")
        print(f"  改善案: {imp['improvement']}")
        print(f"\n🎯 改善理由: {imp['reason']}")
        print(f"✨ 期待効果: {imp['effect']}")
        print("\n" + "=" * 50)

In [8]:
def analyze_accident(report, save_name=''):
    display_report_summary(report)

    # 分析器のインスタンス化
    analyzer = AccidentAnalyzer()

    # 関連する省令の特定
    relevant_reg_result = analyzer.identify_relevant_regulations(report)
    if not relevant_reg_result:
        print("❌ 関連省令の特定に失敗しました")
        return

    # 省令ファイル名の抽出
    reg_file_match = re.search(r'選択されたファイル:\s*(\d+\.md)', relevant_reg_result)
    if not reg_file_match:
        print("❌ 省令ファイル名の抽出に失敗しました")
        return

    # 省令ファイルの読み込み
    regulation_file = reg_file_match.group(1)
    print(f"\n📁 使用する省令ファイル: {regulation_file}")

    try:
        with open(os.path.join('data', regulation_file), 'r', encoding='utf-8') as f:
            regulation_content = f.read()
            print(f"✅ 省令ファイル読み込み完了 ({len(regulation_content)} 文字)")
    except Exception as e:
        print(f"❌ ファイル読み込み時にエラーが発生: {str(e)}")
        return

    # 事故分析の実行
    print("\n🔍 事故分析を開始...")
    analysis_result = analyzer.analyze_accident(report, regulation_content)
    if not analysis_result:
        print("❌ 事故分析に失敗しました")
        return

    # 技術要素の分析
    print("\n🔍 技術要素の分析を開始...")
    technical_analysis = analyzer.identify_technical_elements(report)

    # 省令改善案の生成
    print("\n📋 省令改善案の生成を開始...")
    regulation_improvements = analyzer.generate_regulation_improvements(
        report,
        regulation_content,
        analysis_result
    )

    if regulation_improvements:
        improvements = parse_regulation_improvements(regulation_improvements)
        display_regulation_improvements(improvements)

        # 改善案をCSVファイルとして保存
        improvements_data = []
        for imp in improvements:
            improvements_data.append({
                'proposal_number': imp['number'],
                'target_article': imp['target'],
                'current_regulation': imp['current'],
                'improved_regulation': imp['improvement'],
                'improvement_reason': imp['reason'],
                'expected_effect': imp['effect']
            })

        if improvements_data:
            df = pd.DataFrame(improvements_data)
            os.makedirs('output', exist_ok=True)
            df.to_csv('output/regulation_improvements.csv', index=False, encoding='utf-8')
            print("\n✅ 省令改善案を生成し保存しました")

    # 実践的な問題と解答の生成
    print("\n📝 実践的な問題と解答の生成を開始...")
    if technical_analysis and regulation_content:
        practical_questions = analyzer.generate_practical_questions(
            report,
            regulation_content,
            technical_analysis
        )

        if practical_questions:
            qa_data = parse_practical_questions(practical_questions)
            qa_list = []
            for problem in qa_data:
                qa_list.append({
                    'problem_number': problem['number'],
                    'target_audience': problem['target_audience'],
                    'situation': problem['situation'],
                    'questions': '\n'.join(problem['questions']),
                    'answers': '\n'.join(problem['answers'])
                })
            qa_df = pd.DataFrame(qa_list)

            os.makedirs('output', exist_ok=True)
            qa_df.to_csv(f'output/practical_questions_{save_name}.csv', index=False, encoding='utf-8')
            print("\n✅ 実践的な問題と解答を生成し保存しました")
    else:
        print("❌ 問題生成に必要な情報が不足しています")

In [11]:
# 報告書ファイルの読み込み
with open('data/accident-report.md', 'r', encoding='utf-8') as f:
    content = f.read()

# 報告書の総件数を取得
total_reports = extract_total_reports(content)
if total_reports == 0:
    print("❌ 報告書の総件数を取得できませんでした。")

# 総当たりで報告書を取得して表示
def process_all_reports(content, total_reports, max_iteration):
    """報告書を総当たりで処理"""
    # 報告書を分割してリスト化
    reports = content.split('\n## ')[1:]

    for index, report in enumerate(reports, start=1):
        if max_iteration is not None and index > max_iteration:
            break
        print(f"\n=== 報告書番号: {index} ===")

        report_number = report.split('\n')[0].strip()
        if report_number.startswith('報告書番号: '):
            report_number = report_number[len('報告書番号: '):]

        print(f"処理中の報告書: {report_number} ({index}/{total_reports})")
        full_report = f"## 報告書番号: {report}"
        analyze_accident(full_report, index)

In [12]:
MAX_ITERATION = 10
process_all_reports(content, total_reports, MAX_ITERATION)


=== 報告書番号: 1 ===
処理中の報告書: RI2024-2-1 (1/408)

選択された報告書の内容

📄 報告書番号: 報告書番号: RI2024-2-1

📌 基本情報:
- 発生年月日: 2023年12月12日
- 区分: 軌道
- 発生場所: 山鼻線　中島公園通停留場～山鼻9条停留場間（複線）［北海道札幌市］（すすきの停留場起点1k424m付近）
- 事業者: 公営 - 一般財団法人札幌市交通事業振興公社（法人番号　1430005010801）
- 都道府県: 北海道

🚨 事故詳細:
- 事故等種類: 本線逸走
- 踏切区分: データなし
- 人の死傷: データなし
- 死傷者数: なし

📝 概要:
一般財団法人札幌市交通事業振興公社の山鼻線中央図書館前停留場発内回り循環1両編成の第252号車の運転士は、令和5年12月12日（火）、中島公園通停留場において、停車中の同車両から降車して、同停留場に設置されている連絡電話を使用していたところ、同車両が山鼻9条停留場方向へ動いていることを認めたため、同車両へ駆け寄って乗り込み、ブレーキを使用して同車両を中島公園通停留場から約31mのところで停止させた。
　同車両には乗客21名が乗車していたが、負傷者はいなかった。

🔍 原因:
本重大インシデントは、中島公園通停留場において、停留場の連絡電話を使用するために運転士が降車している状況で、車両のブレーキが緩んだため、停車中の車両が乗客を乗せたまま、下り勾配である山鼻9条停留場方向へ逸走したものと推定される。
　車両のブレーキが緩んだことについては、ブレーキハンドルが「重なり」位置から「緩ゆるめ」位置となったため、車両のブレーキシリンダー圧力が徐々に減圧したことによるものと推定される。
　ブレーキハンドルが「重なり」位置から「緩ゆるめ」位置となったことについては、運転士が着ていた外とうがブレーキハンドルと接触したことによるものと推定され、運転士は外とうを着て厚着であったため、接触した感覚が鈍くなっており、ブレーキハンドルとの接触に気付かなかったものと考えられる。



=== 関連省令の特定結果 ===
選択されたファイル: 08.md  
選択理由:  
事故報告書に記載された内容は、主に運転士が降車中にブレーキが緩んでしまい、停車中の車両が逸走した